In [ ]:
import numpy as np
from banditpy.models import Thompson2Arm
import mab_subjects

exps = mab_subjects.unstruc.allsess

for i, exp in enumerate(exps):

    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)
    choices = task.choices
    choices = choices - 1
    rewards = exp.b2a.rewards

    model = Thompson2Arm(choices=choices, rewards=rewards, n_sim=500)
    result = model.fit()

    print("Fitted Thompson Sampling priors:")
    print(f"  alpha0 = {model.alpha0:.3f}")
    print(f"  beta0  = {model.beta0:.3f}")
    print(f"  Negative Log-Likelihood = {model.nll:.2f}")